In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-15 11:53:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-06-15 11:53:49 (11.5 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

### Deliverable 2 

In [5]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [6]:
# retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_50_df = totalvotes_df.filter("helpful_votes/total_votes>=.50")
helpful_50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [7]:
# Filter the DataFrame and create a new table that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "Y")
paid_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36356690|R184FOUNZZ7KO8|B0117RFX38|     294335899|Bose SoundTrue Ar...|     Electronics|          5|           15|         20|   Y|                N|Transparent - Mus...|When I see the Bo...| 2015-08-31|
|         US|   37291590| R82QWN2X2OCHB|B0117RGG8E|     871919611|Bose SoundLink ar...|     Electronics|          5|    

In [8]:
# retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_vine_review_df = helpful_50_df.filter(helpful_50_df["vine"] == "N")
unpaid_vine_review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14380991|R1FBO737KD9F2N|B00NG57H4S|     213020860|Sony MDRZX110 Noi...|     Electronics|          5|           19|         23|   N|                Y|Great noise cance...|Great pair of hea...| 2015-08-31|
|         US|   51242401|R227GSNWI6BSZV|B00ICNXESC|     111052608|GS Portalac PX120...|     Electronics|          1|    

In [9]:
# Total Number of Reviews
total_reviews = helpful_50_df.count()
total_reviews

50753

In [10]:
# Total of Vine
total_vine = helpful_50_df.filter("vine == 'Y'")
total_vine.count()


1080

In [11]:
# Total of Non-Vine
non_vine = helpful_50_df.filter ("vine == 'N'")
non_vine.count()

49673

In [12]:
# Number of 5-star reviews
total_5star_reviews = helpful_50_df.filter("star_rating == 5")
total_5star_reviews.count()

23497

In [13]:
# Number of 5-star reviews (vine)
vine_reviews = total_vine.filter("star_rating == 5")
vine_reviews.count()

454

In [14]:
# Number of 5-star reviews (non_vine)
non_vine_reviews = non_vine.filter("star_rating == 5")
non_vine_reviews.count()

23043

In [15]:
# 5-Star vine_reviews divided by total_vine
percentage_vine_5star = (vine_reviews.count()/total_vine.count()) * 100 
round(percentage_vine_5star, 2)

42.04

In [17]:
# 5-star vine_reviews divide by total_5star_reviews = percentage of Vine 5-star
percentage_vine_total_5star = (vine_reviews.count()/total_5star_reviews.count()) * 100
round(percentage_vine_total_5star, 2)

1.93

In [18]:
# 5-star non_vine_reviews divided by total non_vine = percentage of non_vine
percentage_nonvine_5star = (non_vine_reviews.count()/non_vine.count()) * 100
round(percentage_nonvine_5star, 2)

46.39

In [19]:
# 5-star non_vine_reviews divide by total_5star_reviews = percentage of non-vine 5-star
percentage_nonvine_total_5star = (non_vine_reviews.count()/total_5star_reviews.count()) * 100
round(percentage_unpaid_total_5star, 2)

98.07